# TicTacToe

For the frist day, we are creating a simple game for two persons. 

In [13]:
# Install ipywidgets and IPython in they are not found
import ipywidgets as widgets
from IPython.display import display


# The game class to construct the tic tac toe game
class TicTacToe: 
    def __init__(self): 
        # self.board = [ 
        #                 [' ', ' ', ' '], 
        #                 [' ', ' ', ' '], 
        #                 [' ', ' ', ' ']
        #             ] 
        self.board = [ [' ' for col in range(3)] for row in range(3)]
        self.player = 'X'
        self.buttons = [ [widgets.Button(description='') for button in range(3)] for row in range(3)]
        
        # register the make_move function to each button's onclick event
        for i in range(3): 
            for j in range(3): 
                self.buttons[i][j].on_click(self.make_move(i, j))
        self.output = widgets.Output()
                
        
    # Put either "X" or "O" on the ith row and jth column
    def make_move(self, i, j): 
        def on_button_clicked(_): 
            if self.board[i][j] == ' ': 
                self.board[i][j] = self.player
                self.buttons[i][j].description = self.player

                # turn taking
                if self.player == 'X': 
                    self.player = 'O'
                else: 
                    self.player = 'X'
        return on_button_clicked
        
    def display(self): 
        grid = widgets.GridBox( [button for row in self.buttons for button in row], 
                                  layout=widgets.Layout( grid_template_columns="repeat(3, 150px)" ) )
        display(grid, self.output)
        
game = TicTacToe()
game.display()

GridBox(children=(Button(style=ButtonStyle()), Button(style=ButtonStyle()), Button(style=ButtonStyle()), Butto…

Output()